# A Longest Path Example -- Building a Stadium

We have the following information about the tasks that need to be completed to build a stadium:


|Task | Duration (weeks) | Predecessors |
|----:|---------:|-------------:|
|1|2|none|
|2|16|1|
|3|9|2|
|4|8|2|
|5|10|3|
|6|6|4,5|
|7|2|4|
|8|2|6|
|9|9|4,6|
|10|5|4|
|11|3|6|
|12|2|9|
|13|1|7|
|14|7|2|
|15|4|4,14|
|16|3|8,11,14|
|17|9|12|
|18|1|17|

What is the shortest time in which we could complete the entire stadium? Which tasks are on the so-called "critical path," or the sequence of activities that will take the longest to complete?

## Building the Data and Model in Julia 

In [1]:
using JuMP, Clp

### DATA ###

# task list
T = [:S, :1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :F]

# each task has a duration
duration = Dict(zip(T,[0,2,16,9,8,10,6,2,2,9,5,3,2,1,7,4,3,9,1,0]))

# the easiest way to build the model is to create a list of predecessors for each task.
# there are multiple was to do this -- I like to assign "1" to a pair of tasks (i,j) if
# task j is preceeded by task i. you can see how i use this in the model below.
using NamedArrays

# initialize a table of which tasks precede others with all 0s
zero_list = zeros(20,20)
pred_list = NamedArray(zero_list, (T,T), ("Task", "Task"))

# assign value of "1" if second task is preceded by fist task. 
# i recommend automating this list creation -- it's very easy to make mistakes
# when doing it by hand!
pred_list[1,2] = 1;  pred_list[2,3] = 1;  pred_list[2,4] = 1;  pred_list[3,5] = 1; pred_list[4,6] = 1; pred_list[5,6] = 1;
pred_list[4,7] = 1; pred_list[6,8] = 1; pred_list[4,9] = 1; pred_list[6,9] = 1; pred_list[4,10] = 1; pred_list[6,11] = 1;
pred_list[9,12] = 1; pred_list[7,13] = 1; pred_list[2,14] = 1; pred_list[4,15] = 1; pred_list[14,15] = 1; 
pred_list[8,16] = 1; pred_list[11,16] = 1; pred_list[14,16] = 1; pred_list[12,17] = 1; pred_list[17,18] = 1; 
pred_list[10,20] = 1; pred_list[13,20] = 1; pred_list[15,20] = 1; pred_list[16,20] = 1; pred_list[18,20] = 1


### MODEL ###

m = Model(Clp.Optimizer)

@variable(m, x[T] >= 0) # variable for the time we begin each task

# create constraint for every pair of tasks (i,j) where task j is preceded by task i
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[j] >= x[i] + duration[i])


# minimize the time we start task F (finish project)
@objective(m, Min, x[:F])

# solve this isntance of the longest path problem
optimize!(m)

# record the value of the variables
xsol = value.(x)

println("Earliest completion time: ", objective_value(m), " weeks")
for i in T
    println("Start task ", i , " in week ", xsol[i])
end


Earliest completion time: 64.0 weeks
Start task S in week 0.0
Start task 1 in week 0.0
Start task 2 in week 2.0
Start task 3 in week 18.0
Start task 4 in week 18.0
Start task 5 in week 27.0
Start task 6 in week 37.0
Start task 7 in week 26.0
Start task 8 in week 44.0
Start task 9 in week 43.0
Start task 10 in week 59.0
Start task 11 in week 43.0
Start task 12 in week 52.0
Start task 13 in week 28.0
Start task 14 in week 18.0
Start task 15 in week 60.0
Start task 16 in week 46.0
Start task 17 in week 54.0
Start task 18 in week 63.0
Start task F in week 64.0
Coin0506I Presolve 0 (-28) rows, 0 (-20) columns and 0 (-56) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 64
Coin0511I After Postsolve, objective 64, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 64 - 0 iterations time 0.012, Presolve 0.01


The only unfortunate thing about solving the problem this way is we now have to do a little more work to discover which tasks are "critical." We can use the "getdual" function to find out which constraints had no slack (the constraints are satisfied at equality in an optimal solution). This tells us which pairs of tasks must be completed in immediate succession to meet our goal of 64 weeks. Other constraints have slack, which means we could delay starting the next task without impacting the total project completion time. 

In [2]:
for i in T
    for j in T
        if pred_list[i,j] == 1
            # this is equivalent to saying "if there is no slack in the constraint"
            if dual(constr[i,j]) > 0
                # then we know this pair of tasks is on the critical path. 
                # to avoid redundancy, we'll only print the predecessor (could also do successor)
                println("Task ", i, " is on the critical path.")
            end
        end
    end
end

Task 1 is on the critical path.
Task 2 is on the critical path.
Task 3 is on the critical path.
Task 5 is on the critical path.
Task 6 is on the critical path.
Task 9 is on the critical path.
Task 12 is on the critical path.
Task 17 is on the critical path.
Task 18 is on the critical path.


Note that if we'd implemented the other formulation of this model, we'd get this information "for free" in the optimal values of the decision variables. This is not a coincidence.... (We'll learn a lot more about duality soon!)